In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool_data_from_file
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
RPC="wss://rpc.hydradx.cloud"
N = 24
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
# buffer2 = {((asset_map[k[0][0]], asset_map[k[0][1]]),k[1]): v for k, v in buffer.items()}

pprint(tokens)


{'ASTR': {'LRNA': 9994.655349057575, 'liquidity': 4235101.758768052},
 'BNC': {'LRNA': 6752.403668986631, 'liquidity': 713862.643678636},
 'CFG': {'LRNA': 9460.865604280176, 'liquidity': 589556.2996413776},
 'DAI': {'LRNA': 12702.251432922867, 'liquidity': 343412.5919105631},
 'DAI001': {'LRNA': 1407.281258338137, 'liquidity': 37585.53464434876},
 'DOT': {'LRNA': 71674.2114941736, 'liquidity': 390750.2853887864},
 'GLMR': {'LRNA': 5831.055420677325, 'liquidity': 610373.0891944987},
 'HDX': {'LRNA': 21466.984615632326, 'liquidity': 106973569.49123076},
 'INTR': {'LRNA': 8594.801432854158, 'liquidity': 13316659.025928365},
 'USDT': {'LRNA': 83678.17738910968, 'liquidity': 2287503.252427},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41090.31683516522, 'liquidity': 31.62994478},
 'WETH': {'LRNA': 21406.107805151998, 'liquidity': 309.5266368689693},
 'WETH001': {'LRNA': 71076.66935212267, 'liquidity': 1026.3435957803392},
 'ZTG': {'LRNA': 4277.134849138

In [5]:
# arb_file = "arbconfig.txt"
arb_file = "arbconfig2.txt"
with open('config/' + arb_file, 'r') as json_file:
    cfg = json.load(json_file)
    
for d in cfg:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_assets = {}
buffer = {}

if get_live_markets:
    for arb_cfg in cfg:
        tkn_pair = arb_cfg['order_book']
        # buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
        exchange = arb_cfg['exchange']
        if exchange not in ob_objs:
            ob_objs[exchange] = {}
        if tkn_pair not in ob_objs[exchange]:
            if exchange == 'kraken':
                ob_objs[exchange][tkn_pair] = get_kraken_orderbook(tkn_pair, archive=True)
            elif exchange == 'binance':
                ob_objs[exchange][tkn_pair] = get_binance_orderbook(tkn_pair, archive=True)
            else:
                raise ValueError("Exchange not supported")
else:
    ob_objs = get_orderbooks_from_file("data/input/")

for arb_cfg in cfg:
    tkn_pair = arb_cfg['order_book']
    exchange = arb_cfg['exchange']
    # buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
    for tkn in tkn_pair:
        if exchange not in order_book_assets:
            order_book_assets[exchange] = []
        if tkn not in order_book_assets[exchange]:
            order_book_assets[exchange].append(tkn) 
            
    arb_cfg['tkn_pair'] = (asset_map[arb_cfg['tkn_ids'][0]], asset_map[arb_cfg['tkn_ids'][1]])
            

                


In [6]:

cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}
# kraken_fee = 0.0016
# buffer = 0.0010

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0

cex_dict = {}
for exchange in ob_objs:
    cex_dict[exchange] = CentralizedMarket(
        order_book=ob_objs[exchange],
        asset_list=order_book_assets[exchange],
        trade_fee=cex_fees[exchange]
    )

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)
init_state = op_state.copy()


In [7]:
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex_dict1, cfg)

pprint(all_swaps1)
pprint(len(all_swaps1))

cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state2 = op_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex_dict2, cfg)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

[{'cex': {'amount': 0.1524547643222809,
          'buy_asset': 'DAI',
          'price': 1887.9165695000002,
          'sell_asset': 'ETH',
          'trade': 'sell'},
  'dex': {'amount': 0.1524547643222809,
          'buy_asset': 'WETH001',
          'max_sell': 285.1556913782358,
          'price': 1888.861,
          'sell_asset': 'DAI001',
          'trade': 'buy'},
  'exchange': 'kraken'},
 {'cex': {'amount': 80.93684577835845,
          'buy_asset': 'ETH',
          'price': 0.0026636675,
          'sell_asset': 'DOT',
          'trade': 'sell'},
  'dex': {'amount': 80.93684577835845,
          'buy_asset': 'DOT',
          'max_sell': 0.21515380972623824,
          'price': 0.002665,
          'sell_asset': 'WETH001',
          'trade': 'buy'},
  'exchange': 'kraken'},
 {'cex': {'amount': 0.004507402484893799,
          'buy_asset': 'XBT',
          'price': 35608.1952,
          'sell_asset': 'USDT',
          'trade': 'buy'},
  'dex': {'amount': 0.004507402484893799,
         

In [8]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list}
for exchange in cex_dict:
    for asset in cex_dict[exchange].asset_list:
        if asset not in holdings:
            holdings[asset] = 1000000000
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}

arb.execute_arb(state1, cex_dict1, agent1, all_swaps1)
arb.execute_arb(state2, cex_dict2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'HDX': 0, 'USD': 3.397627115249634, 'BTC': 0.00010907649993896484, 'ETH': 0.0003052949905395508, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0, 'INTR': 0}
strategy 1 profit at Omnipool spot
7.423840592335538
strategy 2 profit
{'HDX': 0, 'USD': 4.599629163742065, 'BTC': 0.00014483928680419922, 'ETH': 0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0, 'INTR': 0}
strategy 2 profit at Omnipool spot
9.180108399327567
